# Decoerentes' solution to the second question of quantum chemistry

Decoerentes' team - Alisson, Bruno, Carlos, Gabriel, Miguel and Paulo

In [ ]:
!pip install pennylane
!pip install pyscf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 60.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 52.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.7/48.7 MB 11.1 MB/s eta 0:00:00


In [ ]:
import pennylane as qml
from pennylane import numpy as pnp
import matplotlib.pyplot as plt
import pyscf

# 2 a)


In [ ]:
def mol(x1,y1,z1,x2,y2,z2):

  mol= pyscf.M(atom=[["H", (x1, y1, z1)],["H", (x2, y2, z2)]])
  myhf = mol.HF.run()
  myci = myhf.CISD().run()

  # Import the initial state into PennyLane
  wf_cisd = qml.qchem.import_state(myci, tol=1e-1)

  symbols = ["H", "H"]

  geometry = pnp.array([[x1/0.529, y1/0.529, z1/0.529], [x2/0.529, y2/0.529, z2/0.529]])
  molecule = qml.qchem.Molecule(symbols, geometry, charge = 0)
  H2mol, qubits = qml.qchem.molecular_hamiltonian(molecule)
  wires = list(range(qubits))


  electrons = 2
  orbitals = qubits

  # Create all possible excitations in H3+
  singles, doubles = qml.qchem.excitations(electrons, orbitals)
  #combine all the the possible excitations
  excitations = singles + doubles

  dev = qml.device("default.qubit", wires=qubits)
  @qml.qnode(dev)
  def circuit_VQE(theta, initial_state):
      qml.StatePrep(initial_state, wires=wires)
      #construct ansatze
      for i, excitation in enumerate(excitations):
        #if excitation corresponds to a Double excitation, apply a Double Excitation Givens rotation
        if len(excitation) == 4:
             qml.DoubleExcitation(theta[i], wires=excitation)
        #otherwise, apply a Single Excitation Givens rotation
        else:
            qml.SingleExcitation(theta[i], wires=excitation)

      return qml.expval(H2mol)

  opt = qml.GradientDescentOptimizer(stepsize=0.4)
  theta = pnp.array(pnp.zeros(len(excitations)), requires_grad=True)
  #Draw circuit
  qml.draw_mpl(circuit_VQE)( theta, initial_state=wf_cisd);
  delta_E = 10
  tol = 1e-9
# run the VQE optimization loop until convergence threshold is reached
  while abs(delta_E) > tol:
    theta, prev_energy = opt.step_and_cost(circuit_VQE, theta, initial_state=wf_cisd)
    new_energy = circuit_VQE(theta, initial_state=wf_cisd)
    delta_E = new_energy - prev_energy
  return new_energy

# 2b)

In [ ]:

def ground_Ising(h):
  #defining the interaction term sum Z_iZ_i+1#
  O1=0
  for i in range(3):
     O1+= qml.Z(i) @ qml.Z(i+1)
  O1 += qml.Z(3) @ qml.Z(0)
  #defining the linear term sum X_i#
  O2=0
  for i in range(4):
     O2+= qml.X(i)
  O2 =h*O2
  #defining the given hamiltonian#
  coeffs = [-1, -1]
  obs = [O1, O2]
  H = qml.Hamiltonian(coeffs, obs)

  #defines the circuit used to VQE#
  dev = qml.device("default.qubit", 4)
  @qml.qnode(dev)
  def circuitb(theta):
    for i in range(4):
       qml.Hadamard(wires=i)
       qml.RZ(theta[i], wires=i)
       qml.RX(theta[i+4], wires=i)
    #introducing controlled gates to allow quantum correlations#
    for i in range(3):
      qml.CNOT(wires=[i, i+1])
    qml.CNOT(wires=[3,0])
    return qml.expval(H)


  opt = qml.GradientDescentOptimizer(stepsize=0.04)
  theta = pnp.array([0.1,0.1,0.1,0.1,0.1,0.1, 0.1,0.1], requires_grad=True)
  #Draw circuit
  qml.draw_mpl(circuitb)(theta);
  delta_E = 10
  tol = 1e-4
  # run the VQE optimization loop until convergence threshold is reached
  while abs(delta_E) > tol:
    theta, prev_energy = opt.step_and_cost(circuitb, theta)
    new_energy = circuitb(theta)
    delta_E = new_energy - prev_energy

  return new_energy


# 2c)

In [ ]:
dev = qml.device("default.qubit", wires=5)
@qml.qnode(dev)
#calculates the probability of measuring the state |x,y> for a evolution of H= a X@X + b Z@Z, for a given time t and n trotter steps
def circuit_c(a,b,t,n,x,y):
  #loop to aplly all trotter steps

  for i in range(n):
    #add the evolution of exp(-ibtZ@Z/n)
    qml.CNOT(wires=[1,2])
    qml.RZ(-b*t/n,wires=[2])
    qml.CNOT(wires=[1,2])
    #add the evolution of exp(-iatX@X/n)
    qml.Hadamard(1)
    qml.Hadamard(2)
    qml.CNOT(wires=[1,2])
    qml.RZ(-a*t/n,wires=[2])
    qml.CNOT(wires=[1,2])
    qml.Hadamard(1)
    qml.Hadamard(2)
   #add gates to realize swap test for the evolved state with the computational basis
  if x==1:
      qml.X(3)

  if y==1:
      qml.X(4)

  qml.Hadamard(wires=0)
  for i in range(2):
      qml.CSWAP(wires=[0, 1 + i, i+3 ])
  qml.Hadamard(wires=0)

  return qml.expval(qml.Z(0))

#function to give all probabilities at once
def c2(a,b,t,n):
  return circuit_c(a,b,t,n,0,0), circuit_c(a,b,t,n,1,0), circuit_c(a,b,t,n,0,1), circuit_c(a,b,t,n,1,1)